In [1]:
import numpy as np
import librosa
import os

In [5]:
def compute_snr(reference_file, synthesized_file, target_sr=22050):
    """
    Computes the Signal-to-Noise Ratio (SNR) between reference and synthesized audio files.

    Args:
        reference_file (str): Path to the reference audio file.
        synthesized_file (str): Path to the synthesized audio file.
        target_sr (int): Target sample rate for resampling.

    Returns:
        float: The Signal-to-Noise Ratio in dB.
    """
    # Load audio files
    reference, sr_ref = librosa.load(reference_file, sr=None)
    synthesized, sr_syn = librosa.load(synthesized_file, sr=None)

    # Resample if the sample rates are different
    if sr_ref != target_sr:
        reference = librosa.resample(reference, orig_sr=sr_ref, target_sr=target_sr)
    if sr_syn != target_sr:
        synthesized = librosa.resample(synthesized, orig_sr=sr_syn, target_sr=target_sr)

    # Trim signals to the same length
    min_length = min(len(reference), len(synthesized))
    reference = reference[:min_length]
    synthesized = synthesized[:min_length]

    # Calculate signal power and noise power
    noise = synthesized - reference  # Assuming synthesized is the "noisy" signal
    signal_power = np.mean(reference**2)
    noise_power = np.mean(noise**2)

    # Avoid division by zero
    if noise_power == 0:
        return float('inf')  # Infinite SNR if there's no noise

    # Calculate SNR in dB
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

In [6]:
# Directory where your audio files are stored
generated_audio_dir = 'generated_audio/'  # Directory for generated audio files
reference_audio_dir = 'reference_audio/'    # Directory for reference audio files

# List of generated audio files (sample_1.wav to sample_19.wav)
generated_audios = [os.path.join(generated_audio_dir, f'sample_{i}.wav') for i in range(1, 20)]

# List of reference audio files (20.wav to 25.wav)
reference_audios = [os.path.join(reference_audio_dir, f'{i}.wav') for i in range(20, 26)]

# Store SNR scores for each generated audio
snr_scores = {audio: [] for audio in generated_audios}

# Calculate SNR for each generated audio against all reference audios
for generated_audio in generated_audios:
    for reference_audio in reference_audios:
        try:
            snr_score = compute_snr(reference_audio, generated_audio)
            snr_scores[generated_audio].append(snr_score)
        except Exception as e:
            print(f"Error processing {generated_audio} with {reference_audio}: {e}")

# Calculate average SNR for each generated audio across all reference audios
average_snr_scores = {os.path.basename(audio): np.mean(scores) if scores else None for audio, scores in snr_scores.items()}

# Print average SNR for each generated audio
for audio, avg_snr in average_snr_scores.items():
    if avg_snr is not None:
        print(f"Average SNR for {audio}: {avg_snr:.2f} dB")
    else:
        print(f"Could not calculate SNR for {audio}.")

# Calculate the overall average SNR across all generated audios
overall_average_snr = np.mean([score for scores in snr_scores.values() for score in scores if scores])
print(f"\nOverall Average SNR for all generated audios: {overall_average_snr:.2f} dB")

Average SNR for sample_1.wav: -0.40 dB
Average SNR for sample_2.wav: -0.31 dB
Average SNR for sample_3.wav: -0.38 dB
Average SNR for sample_4.wav: -0.41 dB
Average SNR for sample_5.wav: -0.88 dB
Average SNR for sample_6.wav: -0.28 dB
Average SNR for sample_7.wav: -0.42 dB
Average SNR for sample_8.wav: -0.38 dB
Average SNR for sample_9.wav: -0.37 dB
Average SNR for sample_10.wav: -0.34 dB
Average SNR for sample_11.wav: -0.44 dB
Average SNR for sample_12.wav: -0.30 dB
Average SNR for sample_13.wav: -0.37 dB
Average SNR for sample_14.wav: -21.67 dB
Average SNR for sample_15.wav: -0.32 dB
Average SNR for sample_16.wav: -0.36 dB
Average SNR for sample_17.wav: -4.11 dB
Average SNR for sample_18.wav: -0.37 dB
Average SNR for sample_19.wav: -17.07 dB

Overall Average SNR for all generated audios: -2.59 dB
